In [2]:
import pandas as pd
import json

In [4]:
path_cpv_extracted = "data/cpv8_predictions.json"
path_meta = "data/Metadata"
path_data_extracted = "/export/data_ml4ds/NextProcurement/Junio_2025/pliegosPlace_withExtracted/Insiders"

In [9]:
import pyarrow.parquet as pq

#table = pq.read_table(path_data_extracted, use_legacy_dataset=False)
# If it still fails, try:
table = pq.read_table(path_data_extracted, coerce_int96_timestamp_unit=None)


In [11]:
df = table.to_pandas()
df

,place_id,ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode,title,url,id,resultado_tecnico,path_tecnico,resultado_administrativo,path_administrativo,texto_tecnico,texto_administrativo,extracted_objective,generated_objective
0,https://contrataciondelestado.es/sindicacion/l...,[48821000.0],Suministro de dos servidores virtualizados par...,{'administrativo': 'https://contrataciondelest...,1,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,\n \n \n \n \n \n \n \n \n \n \n \n \n \...,Contratación y Patrimonio\n ...,adquisición de dos servidores para maquinas vi...,El objetivo del contrato es la adquisición de ...
1,https://contrataciondelestado.es/sindicacion/l...,[79995000.0],Servicio técnico de apoyo a la migración del c...,{'administrativo': 'https://contrataciondelest...,2,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,\n \n \n \nMINISTERIO \nDE CULTURA \nY DEP...,\n \n \n \nMINISTERIO \nDE CULTURA \nY DEP...,la prestación de un servicio técnico de apoyo ...,El objetivo del contrato es proporcionar servi...
2,https://contrataciondelestado.es/sindicacion/l...,"[[50118110, 60100000]]",Retirada y transporte de los bienes muebles y ...,{'administrativo': 'https://contrataciondelest...,3,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,\n \nSECRETARIA \nDE \nESTADO \nDE \nLA \nSE...,\n \nSECRETARIA \nDE \nESTADO \nDE \nLA \nSE...,el servicio de recogida y transporte de bienes...,El objeto del contrato es proporcionar el serv...
3,https://contrataciondelestado.es/sindicacion/l...,[72600000.0],Servicios De Mantenimiento Del Sistema De Mens...,{'administrativo': 'https://contrataciondelest...,5,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,\nPágina 1 de 12 \nSECRETARÍA DE \nESTADO \n ...,\n \n \nMINISTERIO DE DEFENSA \nSecretaría de...,El objeto del contrato es la contratación de l...,El objetivo es proporcionar un pliego de presc...
4,https://contrataciondelestado.es/sindicacion/l...,[30190000.0],Suministro de talonarios y sellos,{'administrativo': 'https://contrataciondelest...,6,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,6XEGLUHFFLyQGH*HVWLyQGH6HUYLFLRVD&OLHQW...,\nADMINISTRADOR DE INFRAESTRUCTURAS FERROVIAR...,El objeto del contrato es la adquisición de ta...,El objetivo es proporcionar información sobre ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56579,https://contrataciondelestado.es/sindicacion/l...,[31680000.0],Sistema dinámico de adquisición de suministros...,{'administrativo': 'https://contrataciondelest...,63495,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,ILUSTRE AYUNTAMIENTO DE LA\nVILLA DE EL SAUZAL...,AYUNTAMIENTO DE EL SAUZAL\nTENERIFE\nPLIEGO\n ...,La solicitud de adquisición de materiales y eq...,El objetivo de este texto es proporcionar una ...
56580,https://contrataciondelestado.es/sindicacion/l...,[90520000.0],Servicio de gestión de recogida de residuos pe...,{'administrativo': 'https://contrataciondelest...,63496,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,Descargado correctamente,//export/data_ml4ds/NextProcurement/Junio_2025...,\n \nMutua de Andalucía y de Ceuta \n \n \n \...,\n \nPliego Condiciones Particulares Se...,El objeto de este contrato es la prestación de...,El objetivo es proporcionar un servicio integr...
56581,https://contrataciondelestado.es/sindicacion/l...,[71620000.0],Servicio de inspección